In [1]:
import cv2
import os
from os import listdir
from os.path import isfile, join
from random import shuffle
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# img size is 727 x 294

train_data = './specs/_train'
test_data = './specs/_test'

In [2]:
# Returns genre of audio file if genre name is found in filename
def one_hot_label(filename):
    one_hot = np.array([0, 0, 0, 0, 0, 0])
    genres = ['classical', 'hiphop', 'jazz', 'metal', 'reggae', 'pop']
    for i, genre in enumerate(genres):
        if genre in filename:
            one_hot[i] = 1
            return one_hot
    raise ValueError("One or more files is not labeled with a valid genre")

def train_data_with_label():
    train_images = []
    first = True
    for subdir, dirs, files in os.walk(train_data):
        if first == True:
            first = False
            continue
        for i in tqdm(os.listdir(subdir)):
            path = os.path.join(subdir, i)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (121, 49))
            train_images.append([np.array(img), one_hot_label(i)])
    shuffle(train_images)
    return train_images

def test_data_with_label():
    train_images = []
    for i in tqdm(os.listdir(test_data)):
        path = os.path.join(test_data, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (121, 49))
        train_images.append([np.array(img), one_hot_label(i)])
    return train_images


In [3]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

training_images = train_data_with_label()
testing_images = test_data_with_label()

tr_img_data = np.array([i[0] for i in training_images]).reshape(-1, 121, 49, 1)
tr_lbl_data = np.array([i[1] for i in training_images])

tst_img_data = np.array([i[0] for i in testing_images]).reshape(-1, 121, 49, 1)
tst_lbl_data = np.array([[1] for i in testing_images])

Using TensorFlow backend.
100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 316.56it/s]


In [4]:
model = Sequential()

model.add(InputLayer(input_shape=[121,49,1]))
model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(6, activation='softmax'))
optimizer = Adam(lr=1e-3)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=tr_img_data, y=tr_lbl_data, epochs=50, batch_size=100)
model.summary()

Epoch 1/50
540/540 [==============================] - ETA: 4s - loss: 9.7511 - acc: 0.210 - ETA: 3s - loss: 9.2575 - acc: 0.155 - ETA: 2s - loss: 8.1994 - acc: 0.163 - ETA: 1s - loss: 7.0482 - acc: 0.192 - ETA: 0s - loss: 6.3175 - acc: 0.180 - 4s 8ms/step - loss: 6.0066 - acc: 0.1889
Epoch 2/50
540/540 [==============================] - ETA: 3s - loss: 2.6096 - acc: 0.150 - ETA: 2s - loss: 2.3155 - acc: 0.195 - ETA: 1s - loss: 2.2343 - acc: 0.173 - ETA: 0s - loss: 2.1884 - acc: 0.170 - ETA: 0s - loss: 2.1312 - acc: 0.170 - 4s 7ms/step - loss: 2.1005 - acc: 0.1759
Epoch 3/50
540/540 [==============================] - ETA: 2s - loss: 1.7153 - acc: 0.290 - ETA: 2s - loss: 1.6798 - acc: 0.295 - ETA: 1s - loss: 1.7097 - acc: 0.266 - ETA: 0s - loss: 1.7097 - acc: 0.265 - ETA: 0s - loss: 1.7199 - acc: 0.258 - 4s 7ms/step - loss: 1.7202 - acc: 0.2611
Epoch 4/50
540/540 [==============================] - ETA: 3s - loss: 1.6917 - acc: 0.280 - ETA: 2s - loss: 1.6051 - acc: 0.315 - ETA: 1s - loss:

540/540 [==============================] - ETA: 3s - loss: 0.6240 - acc: 0.750 - ETA: 2s - loss: 0.6823 - acc: 0.745 - ETA: 1s - loss: 0.7137 - acc: 0.716 - ETA: 1s - loss: 0.6826 - acc: 0.735 - ETA: 0s - loss: 0.6463 - acc: 0.750 - 4s 8ms/step - loss: 0.6573 - acc: 0.7426
Epoch 30/50
540/540 [==============================] - ETA: 3s - loss: 0.7950 - acc: 0.690 - ETA: 2s - loss: 0.6334 - acc: 0.740 - ETA: 1s - loss: 0.6304 - acc: 0.756 - ETA: 0s - loss: 0.6478 - acc: 0.750 - ETA: 0s - loss: 0.6380 - acc: 0.758 - 4s 7ms/step - loss: 0.6224 - acc: 0.7611
Epoch 31/50
540/540 [==============================] - ETA: 3s - loss: 0.5997 - acc: 0.820 - ETA: 2s - loss: 0.5722 - acc: 0.825 - ETA: 1s - loss: 0.5747 - acc: 0.813 - ETA: 0s - loss: 0.5749 - acc: 0.797 - ETA: 0s - loss: 0.5755 - acc: 0.790 - 4s 7ms/step - loss: 0.5767 - acc: 0.7852
Epoch 32/50
540/540 [==============================] - ETA: 3s - loss: 0.5590 - acc: 0.810 - ETA: 2s - loss: 0.6044 - acc: 0.770 - ETA: 1s - loss: 0.5470 

In [5]:
genres = ['classical', 'hiphop', 'jazz', 'metal', 'pop', 'reggae']
filenames = [name for name in os.listdir(test_data)]

total_samples = len(filenames)
num_correct = 0
second_correct = 0
third_correct = 0

for cnt, data in enumerate(testing_images):
    img = data[0]
    data = img.reshape(1, 121, 49, 1)
    model_out = model.predict([data])
    
    argsort = np.argsort(model_out[0])
#     print(model_out[0])
#     print(argsort)
    
    first, second, third = argsort[-1], argsort[-2], argsort[-3]

    if genres[first] in filenames[cnt]:
#         print(genres[first], filenames[cnt])
        num_correct += 1
    elif genres[second] in filenames[cnt]:
        second_correct += 1
    elif genres[third] in filenames[cnt]:
        third_correct += 1

    formatted_result = """
    '{predicted}' with {probability} for file {filename}
    --------------------------------------------------------------------
    '{second_genre}' with {prob2} is second most likely
    '{third_genre}' with {prob3} is third most likely
    """.format(predicted=genres[first], probability=model_out[0][first], 
               filename=filenames[cnt], second_genre=genres[second],
              prob2=model_out[0][second], third_genre=genres[third],
              prob3=model_out[0][third])

#     print(genres[np.argmax(model_out[0])])
    print(formatted_result, end='')

accuracy = num_correct/total_samples
top2 = (num_correct + second_correct) / total_samples
top3 = (num_correct + second_correct + third_correct) / total_samples

print("""\n    {num_correct} out of {total_samples} classified correctly with {accuracy}% accuracy
    {top2} chance of genre being in top 2 choices
    {top3} chance of genre being in top 3""".format(num_correct=num_correct, total_samples=total_samples, 
                                                    accuracy=accuracy, top2=top2, top3=top3))


    'jazz' with 0.7272131443023682 for file classical.00000.jpg
    --------------------------------------------------------------------
    'classical' with 0.257743775844574 is second most likely
    'pop' with 0.014003010466694832 is third most likely
    
    'classical' with 0.9963393211364746 for file classical.00001.jpg
    --------------------------------------------------------------------
    'jazz' with 0.0036512725055217743 is second most likely
    'pop' with 6.7388582465355285e-06 is third most likely
    
    'jazz' with 0.9232463240623474 for file classical.00002.jpg
    --------------------------------------------------------------------
    'metal' with 0.04196387901902199 is second most likely
    'classical' with 0.028358709067106247 is third most likely
    
    'jazz' with 0.5670574903488159 for file classical.00003.jpg
    --------------------------------------------------------------------
    'classical' with 0.43222561478614807 is second most likely
    'pop'

    
    'metal' with 0.9998804330825806 for file metal.00002.jpg
    --------------------------------------------------------------------
    'hiphop' with 7.672579522477463e-05 is second most likely
    'pop' with 3.884575198753737e-05 is third most likely
    
    'metal' with 0.996250569820404 for file metal.00003.jpg
    --------------------------------------------------------------------
    'jazz' with 0.0026943436823785305 is second most likely
    'hiphop' with 0.0008309168624691665 is third most likely
    
    'metal' with 0.9876169562339783 for file metal.00004.jpg
    --------------------------------------------------------------------
    'hiphop' with 0.008259650319814682 is second most likely
    'pop' with 0.0036228024400770664 is third most likely
    
    'metal' with 0.9997277855873108 for file metal.00005.jpg
    --------------------------------------------------------------------
    'hiphop' with 0.00019980609067715704 is second most likely
    'pop' with 5.68318